In [9]:
!pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   --- ------------------------------------ 3.4/39.5 MB 15.5 MB/s eta 0:00:03
   --------- ------------------------------ 9.2/39.5 MB 22.0 MB/s eta 0:00:02
   -------------- ------------------------- 14.4/39.5 MB 22.6 MB/s eta 0:00:02
   -------------------- ------------------- 19.9/39.5 MB 23.3 MB/s eta 0:00:01
   -------------------------- ------------- 26.0/39.5 MB 24.2 MB/s eta 0:00:01
   -------------------------------- ------- 32.0/39.5 MB 24.8 MB/s eta 0:00:01
   -------------------------------------- - 38.0/39.5 MB 25.4 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.5 MB 25.5 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.5 MB 25.5 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 19.3 MB/s eta 0:00:00


# Stap 1: Data voorbereiden:
1. Zorg dat je een dataset hebt van ruim meer dan 1000 afbeeldingen
2. Splits de data in 3 datasets:

    a) x_train: ongeveer 80% van de gehele dataset
    
    b) x_test: ongeveer 20% van de gehele dataset
    
    c) x_val: ongeveer 5 afbeeldingen die geen anomally zijn (uit je dataset) + een aantal anomallies
<b> Belangrijk </b>, zorg er voor data je de data _random_ split

3. Zet de afbeeldingen om naar 28 x 28
4. Maak de afbeeldingen zwart wit
5. Maak van elke dataset een numpy object met de shape (aantal_afbeeldingen, 28, 28)

<b> Tip: </b> Kijk even of het is goedgegaan door door je datasets heen te bladeren en afbeeldingen met de "imshow" functie van matplotlib

6. Pixels hebben normaal een waarde tussen 0 en 255, het model werkt alleen met waarden tussen 0 en 1

In [10]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir("Subsample_image_data")
# of os.chdir(‘image_data’)
os.listdir()

image_name = '00001_3_F.png'
image_name

image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
# image.shape
image = Image.fromarray(image)
image


# resize de image
image = np.array((image.resize((28,28), Image.LANCZOS)))
image.shape


AttributeError: 'NoneType' object has no attribute '__array_interface__'

# Stap 2: Het trainen van de Autoencoder
Hieronder krijgen jullie de code van het model (de autoencoder), hij is opgesschreven als functie. In de documentatie van de functie staat wat de inputs en outputs zijn.

In [ ]:
def train_autoencoder(x_train, x_test, latent_dim, num_epochs):
    """
    :param x_train: this should be (around) 80% of your train dataset !!important, use a random split
    :param x_test: this should be (around) 20 % of your train dataset !!important, use a random split
    :param latent_dim: This is the space the images will be reduced to, to start: use the value 164, play with it later
    :param num_epochs: The number of training rounds
    :return: [0] : the trained autoencoder model
             [1] : the training data loss per epoch
             [2] : the test data loss per epoch
    """
    import tensorflow as tf
    from tensorflow.keras import layers, losses
    from tensorflow.keras.models import Model

    class Autoencoder(Model):
        def __init__(self, latent_dim):
            super(Autoencoder, self).__init__()
            self.latent_dim = latent_dim
            self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                layers.Dense(latent_dim, activation='relu'),
            ])
            self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
            ])

        def call(self, x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return decoded

    autoencoder = Autoencoder(latent_dim)

    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

    history = autoencoder.fit(x_train, x_train,
                    epochs=num_epochs,
                    shuffle=True,
                    validation_data=(x_test, x_test))

    return autoencoder, history.history['loss'], history.history['val_loss']

## 2.1 Trainen
Train het model met de functie hierboven, kies een lage waarde voor de epochs

## 2.2 Analyseer de loss functie:
Plot de loss functie en bepaal of je het model lang genoeg getrained hebt

# Stap 3: Predictions
Gebruik de predict method van het model en stop hier je x_val dataset in.
Plot de originele en gereconstrueerde afbeeldingen en vergelijk ze, zie je verschil met de outliers?

# Stap 4: Outlier detection
## 4.1 Quanitificeer de verschillen
Vergelijk de verschillen in de afbeeldingen met verschillende performance metrics zoals MSE, RMSE, MAPE, 
gebruik hiervoor de module <b> sklearn.metrics

Kies eerst MSE om te kijken of er een significant verschill is met outliers en niet-outliers

# 4.2 Tunen
Kan je alle outliers eruit halen op basis van de metrics? Gefeliciteerd
Mocht dit niet het geval zijn, dan kan dit aan de "latent_dim" liggen, de ruimte naar waar de afbeelding wordt samengeknepen in het model, speel met deze parameter en kijk of het de verschillen tussen de outliers en niet-outliers kan verbeteren.

## 4.3 Threshold
Als de vorige stappen goed werken wordt het tijd om automatisch aan te geven wat de outliers zijn.
Kies een threshold voor de door jouw gekozen performance metric.

# Opdracht voor volgende week
Maak een algoritme dat automatisch verteld of een afbeelding een anomally is op basis van jouw model.

Deel daarnaast mogelijke interessante bevindingen.

Iedereen mag even kort voor de groep presenteren